In [53]:
# importings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [54]:
#importing raw data
covid_raw = pd.read_csv('../Data/Raw/owid-covid-data.csv')

In [55]:
covid_raw.head(5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [56]:
print(covid_raw.columns)

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [57]:
covid_new = covid_raw.drop(['new_cases_smoothed','new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million',
       'icu_patients_per_million',
       'hosp_patients_per_million',
       'weekly_icu_admissions_per_million',
       'weekly_hosp_admissions_per_million',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'new_vaccinations_smoothed',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'new_vaccinations_smoothed_per_million',
       'new_people_vaccinated_smoothed_per_hundred','excess_mortality_cumulative_per_million'], axis=1)

In [58]:
# We want to use data's until there are no new cases and deaths in a country (2023-07-02)
# USA didn't submit new cases since 2023-05-21
covid_new['date'] = pd.to_datetime(covid_new['date'])
covid_new = covid_new[covid_new['date'] < '2023-05-20']

In [59]:
# understandig missing datas:
nan_columns = covid_new.columns[covid_new.isna().any()].tolist()
nan_columns

['continent',
 'total_cases',
 'new_cases',
 'total_deaths',
 'new_deaths',
 'reproduction_rate',
 'icu_patients',
 'hosp_patients',
 'weekly_icu_admissions',
 'weekly_hosp_admissions',
 'total_tests',
 'new_tests',
 'positive_rate',
 'tests_per_case',
 'tests_units',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_people_vaccinated_smoothed',
 'stringency_index',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_thousand',
 'life_expectancy',
 'human_development_index',
 'excess_mortality_cumulative_absolute',
 'excess_mortality_cumulative',
 'excess_mortality']

now we are going to understand each column missing data and handle it:

In [60]:
# There are a lot of missing data for countries: England, Hong Kong, Macao, Northern Cyprus, Northern Ireland, Scotland, Taiwan, Wales
missing_data_locations = ['England', 'Hong Kong', 'Macao', 'Northern Cyprus', 'Northern Ireland', 'Scotland', 'Taiwan', 'Wales']
covid_new = covid_new[~covid_new['location'].isin(missing_data_locations) ]

In [61]:
# continent:
covid_new[covid_new['continent'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality
1506,OWID_AFR,NaN,Africa,2020-01-05,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN
1507,OWID_AFR,NaN,Africa,2020-01-06,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN
1508,OWID_AFR,NaN,Africa,2020-01-07,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN
1509,OWID_AFR,NaN,Africa,2020-01-08,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN
1510,OWID_AFR,NaN,Africa,2020-01-09,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375788,OWID_WRL,NaN,World,2023-05-15,765587065.0,0.0,6937640.0,0.0,NaN,NaN,...,6.434,34.635,60.13,2.705,72.58,0.737,7.975105e+09,NaN,NaN,NaN
375789,OWID_WRL,NaN,World,2023-05-16,765587065.0,0.0,6937640.0,0.0,NaN,NaN,...,6.434,34.635,60.13,2.705,72.58,0.737,7.975105e+09,NaN,NaN,NaN
375790,OWID_WRL,NaN,World,2023-05-17,765587065.0,0.0,6937640.0,0.0,NaN,NaN,...,6.434,34.635,60.13,2.705,72.58,0.737,7.975105e+09,NaN,NaN,NaN
375791,OWID_WRL,NaN,World,2023-05-18,765587065.0,0.0,6937640.0,0.0,NaN,NaN,...,6.434,34.635,60.13,2.705,72.58,0.737,7.975105e+09,NaN,NaN,NaN


<div dir=rtl >
<font  size=3>
در اینجا متوجه می‌شویم که اون‌هایی که ستون قاره مربوطه نال است، نام قاره در ستون مکان آمده است، حال اول چک کنیم که تعداد مبتلا‌های کشور‌های با خود قاره برابر است یا نه.
</font>
</div>

In [62]:
africa_datas = covid_new[covid_new['continent'] == 'Africa']
africa_continent_datas = covid_new[covid_new['location'] == 'Africa']
africa_datas[africa_datas['date'] == '2024-02-18']['total_cases'].sum() == africa_continent_datas[africa_continent_datas['date'] == '2024-02-18']['total_cases']

Series([], Name: total_cases, dtype: bool)

<div dir=rtl >
<font  size=3>
متوجه می‌شویم که داده‌هایی که ستون قاره آن ها خالی است تنها آمده است کشورها را جمع زده است. پس سطر‌های مربوط را پاک می‌کنیم.
</font>
</div>

In [63]:
covid_new = covid_new.drop(covid_new[covid_new['continent'].isna()].index, axis=0)

In [64]:
covid_new[covid_new['continent'].isna()].count

<bound method DataFrame.count of Empty DataFrame
Columns: [iso_code, continent, location, date, total_cases, new_cases, total_deaths, new_deaths, reproduction_rate, icu_patients, hosp_patients, weekly_icu_admissions, weekly_hosp_admissions, total_tests, new_tests, positive_rate, tests_per_case, tests_units, total_vaccinations, people_vaccinated, people_fully_vaccinated, total_boosters, new_vaccinations, new_people_vaccinated_smoothed, stringency_index, population_density, median_age, aged_65_older, aged_70_older, gdp_per_capita, extreme_poverty, cardiovasc_death_rate, diabetes_prevalence, female_smokers, male_smokers, handwashing_facilities, hospital_beds_per_thousand, life_expectancy, human_development_index, population, excess_mortality_cumulative_absolute, excess_mortality_cumulative, excess_mortality]
Index: []

[0 rows x 43 columns]>

In [65]:
# total_cases
covid_new[covid_new['total_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379176,ZWE,Africa,Zimbabwe,2020-03-17,NaN,0.0,5.0,0.0,NaN,NaN,...,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN
379177,ZWE,Africa,Zimbabwe,2020-03-18,NaN,0.0,5.0,0.0,NaN,NaN,...,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN
379178,ZWE,Africa,Zimbabwe,2020-03-19,NaN,0.0,5.0,0.0,NaN,NaN,...,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN
379179,ZWE,Africa,Zimbabwe,2020-03-20,NaN,0.0,5.0,0.0,NaN,NaN,...,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN


In [66]:
# we can see that when there is no cases the total_cases is NaN so we replace the NaN with 0:
covid_new['total_cases'] = covid_new['total_cases'].fillna(0)

In [67]:
covid_new[covid_new['total_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality


In [68]:
# new_cases
covid_new[covid_new['new_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality
13554,ARG,South America,Argentina,2020-01-01,0.0,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN
13555,ARG,South America,Argentina,2020-01-02,0.0,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN
13556,ARG,South America,Argentina,2020-01-03,0.0,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN
13557,ARG,South America,Argentina,2020-01-04,0.0,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN
34033,BLZ,North America,Belize,2022-04-10,57287.0,NaN,664.0,8.0,0.75,NaN,...,NaN,NaN,90.083,1.30,74.62,0.716,405285.0,NaN,NaN,NaN
92277,ECU,South America,Ecuador,2020-09-13,116451.0,NaN,10864.0,4140.0,1.06,NaN,...,2.0,12.3,80.635,1.50,77.01,0.759,18001002.0,35103.50000,64.65,38.93
98707,GNQ,Africa,Equatorial Guinea,2022-03-20,15800.0,NaN,183.0,0.0,0.40,NaN,...,NaN,NaN,24.640,2.10,58.74,0.592,1674916.0,NaN,NaN,NaN
135722,GTM,North America,Guatemala,2020-04-05,50.0,NaN,1.0,0.0,NaN,NaN,...,NaN,NaN,76.665,0.60,74.30,0.663,17843914.0,-1140.00000,-5.00,-20.96
199308,MWI,Africa,Malawi,2022-11-06,88009.0,NaN,2685.0,2.0,0.12,NaN,...,4.4,24.7,8.704,1.30,64.26,0.483,20405318.0,NaN,NaN,NaN
201392,MDV,Asia,Maldives,2020-03-29,16.0,NaN,NaN,0.0,NaN,NaN,...,2.1,55.0,95.803,NaN,78.92,0.740,523798.0,NaN,NaN,NaN


In [69]:
# It seems that these remaining NaN data for new cases are 0 at the beginning of the pandemic
covid_new['new_cases'] = covid_new['new_cases'].fillna(0)

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality
13554,ARG,South America,Argentina,2020-01-01,0.0,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN
13555,ARG,South America,Argentina,2020-01-02,0.0,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN
13556,ARG,South America,Argentina,2020-01-03,0.0,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN
13557,ARG,South America,Argentina,2020-01-04,0.0,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN
34033,BLZ,North America,Belize,2022-04-10,57287.0,NaN,664.0,8.0,0.75,NaN,...,NaN,NaN,90.083,1.30,74.62,0.716,405285.0,NaN,NaN,NaN
92277,ECU,South America,Ecuador,2020-09-13,116451.0,NaN,10864.0,4140.0,1.06,NaN,...,2.0,12.3,80.635,1.50,77.01,0.759,18001002.0,35103.50000,64.65,38.93
98707,GNQ,Africa,Equatorial Guinea,2022-03-20,15800.0,NaN,183.0,0.0,0.40,NaN,...,NaN,NaN,24.640,2.10,58.74,0.592,1674916.0,NaN,NaN,NaN
135722,GTM,North America,Guatemala,2020-04-05,50.0,NaN,1.0,0.0,NaN,NaN,...,NaN,NaN,76.665,0.60,74.30,0.663,17843914.0,-1140.00000,-5.00,-20.96
199308,MWI,Africa,Malawi,2022-11-06,88009.0,NaN,2685.0,2.0,0.12,NaN,...,4.4,24.7,8.704,1.30,64.26,0.483,20405318.0,NaN,NaN,NaN
201392,MDV,Asia,Maldives,2020-03-29,16.0,NaN,NaN,0.0,NaN,NaN,...,2.1,55.0,95.803,NaN,78.92,0.740,523798.0,NaN,NaN,NaN


In [76]:
index_zero = covid_new[covid_new['new_cases'] == 0].index
covid_new.loc[covid_new.index == index_zero]


ValueError: Lengths must match to compare